In [1]:
import requests
import json
import time
import copy
import re
import math
import unicodedata
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import time
import datetime

In [2]:
def web_request(method_name, url, dict_data, is_urlencoded=True):
    """Web GET or POST request를 호출 후 그 결과를 dict형으로 반환 """
    method_name = method_name.upper()  # 메소드이름을 대문자로 바꾼다
    if method_name not in ('GET', 'POST'):
        raise Exception('method_name is GET or POST plz...')

    if method_name == 'GET':  # GET방식인 경우
        response = requests.get(url=url, params=dict_data,headers={'Content-Type': 'application/x-www-form-urlencoded'
                                              , 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.26'
                                              })
    elif method_name == 'POST':  # POST방식인 경우
        if is_urlencoded is True:
            response = requests.post(url=url, data=dict_data,
                                     headers={'Content-Type': 'application/x-www-form-urlencoded'
                                              , 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.26'
                                              }
                                    )
        else:
            response = requests.post(url=url, data=json.dumps(dict_data), headers={'Content-Type': 'application/json'})

    dict_meta = {'status_code': response.status_code, 'ok': response.ok, 'encoding': response.encoding,
                 'Content-Type': response.headers['Content-Type']}
    if 'json' in str(response.headers['Content-Type']):  # JSON 형태인 경우
        return {**dict_meta, **response.json()}
    else:  # 문자열 형태인 경우
        return {**dict_meta, **{'text': response.text}}

In [3]:
def getReserveMainInfoByHtml(list_html):
    reserve_info_list = []
    for item in list_html.select('div.tableA tr'):
            #서비스명	장소명	접수기간/이용기간	이용요금	접수상태	신청방법	예약방법
            #print(inside_item.select('div.sch-rslt'))
            if len(item.select('td')) > 0:
                reserve_info = {}
                reserve_id = item.select_one('a')['onclick']
                reserve_id = reserve_id[reserve_id.index("('")+2:reserve_id.rindex("')")]
                reserve_info['예약ID'] = reserve_id
                # 서비스명, 장소명
                reserve_info['서비스명'] = item.select('div.sch-rslt')[0].text
                reserve_info['장소명'] = item.select('div.sch-rslt')[1].text
                ## 접수기간/이용기간
                reserveTIme = item.select_one('td.tbl_date').text
                reserve_info['접수기간'] = reserveTIme[reserveTIme.find(' ', 20, 30):].strip()
                reserve_info['이용기간'] = reserveTIme[:reserveTIme.find(' ', 20, 30)].strip()
                ## 이용요금, 접수상태
                reserve_info['이용요금'] = item.select('td.tbl_label span.bd_label')[0].text.strip()
                reserve_info['접수상태'] = item.select('td.tbl_label span.bd_label')[1].text.strip()
                ##신청방법(오류날 확률이 젤 높음)
                reserve_info['신청방법'] = item.select('td')[5].text.strip()
                ##예약방법
                reserve_info['예약방법'] = item.select_one('span.bd_ico').text.strip()
                if '외부사이트' in reserve_info['예약방법']:
                    reserve_info['외부링크여부'] = 'Y'
                    reserve_info['외부링크'] = reserve_id
                else:
                    reserve_info['외부링크여부'] = 'N'
                    reserve_info['외부링크'] = None
                reserve_info_list.append(reserve_info)
    return reserve_info_list

In [4]:
def getReserveInfoWithOutKeyName(): ## for backup
    for item in list_html.select('div.tableA tr'):
        if len(item.select('td')) > 0:
            #print('#서비스명	장소명	접수기간/이용기간	이용요금	접수상태	신청방법	예약방법')
            start_num = 0
            for inside_item in item.select('td'):
                if start_num == 0 :
                    reserve_id = inside_item.select_one('a')['onclick']
                    reserve_id = reserve_id[reserve_id.index("('")+2:reserve_id.rindex("')")]
                else :
                    print(inside_item.text.strip().replace('\n',','))
                start_num +=1

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [5]:
def 예약전체리스트만들기(req_method, url, body):
    def getReserveMainInfoByHtml(list_html):
        reserve_info_list = []
        for item in list_html.select('div.tableA tr'):
                #서비스명	장소명	접수기간/이용기간	이용요금	접수상태	신청방법	예약방법
                #print(inside_item.select('div.sch-rslt'))
                if len(item.select('td')) > 0:
                    reserve_info = {}
                    reserve_id = item.select_one('a')['onclick']
                    reserve_id = reserve_id[reserve_id.index("('")+2:reserve_id.rindex("')")]
                    reserve_info['_id'] = 'ObjectID()'
                    reserve_info['예약ID'] = reserve_id
                    # 서비스명, 장소명
                    reserve_info['서비스명'] = item.select('div.sch-rslt')[0].text
                    reserve_info['장소명'] = item.select('div.sch-rslt')[1].text
                    ## 접수기간/이용기간
                    reserveTIme = item.select_one('td.tbl_date').text
                    reserve_info['접수기간'] = reserveTIme[reserveTIme.find(' ', 20, 30):].strip()
                    reserve_info['이용기간'] = reserveTIme[:reserveTIme.find(' ', 20, 30)].strip()
                    ## 이용요금, 접수상태
                    reserve_info['이용요금'] = item.select('td.tbl_label span.bd_label')[0].text.strip()
                    reserve_info['접수상태'] = item.select('td.tbl_label span.bd_label')[1].text.strip()
                    ##신청방법(오류날 확률이 젤 높음)
                    reserve_info['신청방법'] = item.select('td')[5].text.strip()
                    ##예약방법
                    reserve_info['예약방법'] = item.select_one('span.bd_ico').text.strip()
                    if '외부사이트' in reserve_info['예약방법']:
                        reserve_info['외부링크여부'] = 'Y'
                        reserve_info['외부링크'] = reserve_id
                    else:
                        reserve_info['외부링크여부'] = 'N'
                        reserve_info['외부링크'] = None
                    reserve_info_list.append(reserve_info)
        return reserve_info_list

    first_res = web_request(req_method, url, body)
    first_html = bs(first_res['text'], "html.parser")
    센터정보리스트 = []
    #list_labels = ['서비스명','장소명',	접수기간/이용기간	이용요금	접수상태	신청방법	예약방법]
    total_pages = math.ceil(int(first_html.select_one('h3.title_dep1 span.text_red').text)/6)
    센터정보리스트 = getReserveMainInfoByHtml(first_html)
    print('1페이지 센터정보생성')
    
    for idx in tqdm(range(2,total_pages+1)):
        res = web_request(req_method, url+'?currentPage='+str(idx), body)
        html = bs(res['text'], "html.parser")
        temp = getReserveMainInfoByHtml(html)
        센터정보리스트.extend(temp)
        print(str(idx)+'페이지 센터정보생성')
        
    return 센터정보리스트

In [6]:
def getDetailInfo(detail_html):
    center_info = {}
    center_info['이미지경로']= 'https://yeyak.seoul.go.kr/' + detail_html.select_one('div.left_box div.img_box img')['src']
    for item in detail_html.select('div.con_box ul.dt_top_list li'):
        if len(item.select('b')) == 0 : 
            continue
        key = item.select('b')[0].text
        value= item.prettify()
        value = cleanhtml(value[value.index('</b>')+4:])
        value = value.strip().replace('\t', '').replace('\n', '').replace('\r', ' ')
        center_info[key] = value
    # 콘텐츠의 외부좌표가 존재하지 않는 경우도 있음
    for tag in detail_html.find_all('input', type='hidden', attrs={'name': 'x'}):
        if tag['value'].replace(' ', '') != '':
            center_info['경도'] = float(tag['value'])
        #else:        

    for tag in detail_html.find_all('input', type='hidden', attrs={'name': 'y'}):
        if tag['value'].replace(' ', '') != '':
            center_info['위도'] = float(tag['value'])
    #for tag in detail_html.find_all('input', type='hidden', attrs={'name': 'mgis_cdata'}):
    return center_info

In [7]:
LIST_PAGE = 'https://yeyak.seoul.go.kr/web/search/selectPageListDetailSearch.do'
IMG_LIST_PAGE = 'https://yeyak.seoul.go.kr/web/search/selectPageListDetailSearchImg.do'
DETAIL_PAGE = 'https://yeyak.seoul.go.kr/web/reservation/selectReservView.do'
RESERVE_CAL = 'https://yeyak.seoul.go.kr/web/reservation/selectListReservCalAjax.do'
REQ_GET = 'GET'
REQ_POST = 'POST'
now = datetime.datetime.now()
today_yyyymmdd = now.strftime("%Y%m%d")

In [8]:
total_list = []
# 체육시설, 문화체험, 공간시설, 교육강좌, 진료복지
#code_list = ['T100', 'T200', 'T500', 'T000', 'T400']
code_list = ['T200','T000']
for code in code_list:
    total_list.extend(예약전체리스트만들기(REQ_POST,LIST_PAGE,{'code':code}))

1페이지 센터정보생성


  1%|          | 1/114 [00:00<01:04,  1.76it/s]

2페이지 센터정보생성


  2%|▏         | 2/114 [00:01<01:07,  1.66it/s]

3페이지 센터정보생성


...

96페이지 센터정보생성


 99%|█████████▉| 96/97 [00:52<00:00,  1.87it/s]

97페이지 센터정보생성


100%|██████████| 97/97 [00:53<00:00,  1.82it/s]

98페이지 센터정보생성


In [13]:
total_list[364]

{'_id': 'ObjectID()',
 '예약ID': 'S230828110130558131',
 '서비스명': "푸른수목원 10월 숲해설 '열매와 씨앗이야기'(단체)",
 '장소명': '푸른수목원(구로구)',
 '접수기간': '2023.10.01 ~ 2023.10.31',
 '이용기간': '2023.09.18 ~ 2023.10.31',
 '이용요금': '무료',
 '접수상태': '예약마감',
 '신청방법': '선착순',
 '예약방법': '온라인',
 '외부링크여부': 'N',
 '외부링크': None}

In [12]:
for item in tqdm(total_list):
    if item['외부링크여부'] == 'N' and '일시정지' not in item['접수상태'] and '예약마감' not in item['접수상태']:
        detail_res = web_request(REQ_POST, DETAIL_PAGE, {'sysToday': today_yyyymmdd, 'rsv_svc_id': item['예약ID']})
        detail_html = bs(detail_res['text'], "html.parser")
        item.update(getDetailInfo(detail_html))

 29%|██▊       | 364/1273 [01:27<03:39,  4.14it/s]


TypeError: 'NoneType' object is not subscriptable

In [32]:
total_list[1140]

{'_id': 'ObjectID()',
 '예약ID': 'http://dongdaemun.kccf.or.kr/home/main/school.php?mode=view&menuinfo_code=lecture&page=1&content_idx=43162&keyword=&keyfield=',
 '서비스명': '이현주 힐링 노래교실',
 '장소명': '(동대문구)',
 '접수기간': '',
 '이용기간': '~   ~',
 '이용요금': '무료',
 '접수상태': '접수중',
 '신청방법': '-',
 '예약방법': '외부사이트',
 '외부링크여부': 'Y',
 '외부링크': 'http://dongdaemun.kccf.or.kr/home/main/school.php?mode=view&menuinfo_code=lecture&page=1&content_idx=43162&keyword=&keyfield='}

In [33]:
for item in tqdm(total_list):
    if item['접수기간'] == '':
        continue
    reverve_info = {}
    str_dt = item['접수기간'].split('~')[0]
    if len(str_dt.split(' ')) > 1:
        reverve_info['접수시작일'] = str_dt.split(' ')[0]
        reverve_info['접수시작시간'] = str_dt.split(' ')[1]    
    else:
        reverve_info['접수시작일']=str_dt
    #접수시작일 = str_dt.split(' ')[0]
    #접수시작시간 = str_dt.split(' ')[1]
    end_dt= item['접수기간'].split('~')[1]
    end_dt = end_dt.lstrip()
    #접수종료일 = end_dt.split(' ')[0]
    #접수종료시간 = end_dt.split(' ')[1]
    if len(end_dt.split(' ')) > 1:
        reverve_info['접수종료일'] = end_dt.split(' ')[0]
        reverve_info['접수종료시간'] = end_dt.split(' ')[1]    
    else:
        reverve_info['접수시작일']=end_dt
    item.update(reverve_info)

100%|██████████| 1273/1273 [00:00<00:00, 456823.15it/s]


In [ ]:
total_list

In [39]:
with open('./서울예약정보_20230919.json', 'w', encoding='utf-8') as make_file:
    json.dump(total_list, make_file, indent="\t")